In [53]:
import pandas as pd
from bs4 import BeautifulSoup
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
import requests
import os
from splinter import Browser

In [54]:
executable_path = {'executable_path' : ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\tbrid\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [8]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')


In [16]:
mars = {}
title = soup.find_all('div',class_='content_title')[1].a.text
article = soup.find_all('div', class_='article_teaser_body')[0].text
mars['news title'] = title
mars['news article'] = article
mars

{'news title': "NASA's Curiosity Rover Captures Shining Clouds on Mars",
 'news article': 'The science team is studying the clouds, which arrived earlier and formed higher than expected, to learn more about the Red Planet. '}

In [17]:
url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [22]:
image = soup.find('div', class_='header')
link = image.find('img', class_='headerimage')['src']
print(link)

image/featured/mars1.jpg


In [28]:
base_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/"
featured_image_url = base_url+link
print(image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


In [37]:
space_url = "https://space-facts.com/mars/"
space_table = pd.read_html(space_url)[0]
space_table.columns = ['Specs', 'Mars']
space_table.set_index('Specs', inplace=True)
mars_table = space_table.to_html()


In [79]:
mars_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [80]:
results = soup.find_all('div', class_='description')
image_urls = []
for name in results:
    mars_dic = {'title':[],'image_url':[]}
    title = name.find('h3').text
    mars_dic['title'].append(title)
    browser.click_link_by_partial_text(title)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    img = soup.find_all('img')[5]['src']
    base_url = 'https://astrogeology.usgs.gov'
    mars_dic['image_url'].append(base_url + img)
    image_urls.append(mars_dic)
    browser.back()
    
image_urls
        

[{'title': ['Cerberus Hemisphere Enhanced'],
  'image_url': ['https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg']},
 {'title': ['Schiaparelli Hemisphere Enhanced'],
  'image_url': ['https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg']},
 {'title': ['Syrtis Major Hemisphere Enhanced'],
  'image_url': ['https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg']},
 {'title': ['Valles Marineris Hemisphere Enhanced'],
  'image_url': ['https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']}]